# Construct datasets

In [44]:
import csv
import json
import random as rd


import pandas as pd
from surprise import (
    Dataset,
    Reader,
    KNNBasic,
)
from surprise.model_selection import (
    train_test_split,
)
from surprise.accuracy import rmse

In [45]:
def load_reviews(path: str):
    """
    read user-item rating
    :param path: 
    :return: 
    """
    review_dict = {
        'itemID': [],
        'userID': [],
        'rating': []
    }
    with open(path, 'r', encoding='utf-8') as f:
        csv_reader = csv.reader(f)
        for i, row in enumerate(csv_reader):
            if not i:
                continue
            review_dict['itemID'].append(row[0])
            review_dict['userID'].append(row[1])
            is_recommended = row[2]
            review_dict['rating'].append(2 if is_recommended == 'True' else 1)
    df = pd.DataFrame(review_dict)
    return df


def load_games(path: str):
    """
    Read game information
    :param path: 
    :param positive_ratio_threshold: 
    :return: 
    """
    ret = []
    with open(path, 'r', encoding='utf-8') as f:
        csv_reader = csv.reader(f)
        for i, row in enumerate(csv_reader):
            if not i:
                continue
            ret.append(
                {
                    'app_id': row[0],
                    'title': row[1],
                    'rating': row[-1],
                }
            )
    return ret

In [46]:
reviews_df = load_reviews('data/user_recommendations_subset.csv')
games = load_games('data/game_details_subset.csv')
reviews_df = reviews_df[
    reviews_df['itemID'].isin([game['app_id'] for game in games])
]
reader = Reader(rating_scale=(1, 2))
reviews_dataset = Dataset.load_from_df(
    reviews_df[['userID', 'itemID', 'rating']],
    reader
)
train, test = train_test_split(reviews_dataset, test_size=0.2)

In [47]:
print('[games] len: {}'.format(len(games)))

reviews_df

[games] len: 50


,itemID,userID,rating
0,431960,8588767,2
1,374320,3635265,1
2,292030,9320019,2
3,550,7609653,2
4,292030,8543862,2
...,...,...,...
604,377160,6612917,2
605,292030,13326161,2
606,377160,13326161,2
607,377160,5433546,2


# Build the training model

In [48]:
N_NEIGHBORS = 5

sim_options = {
    "name": "cosine",  # sim_options(dict): A dictionary of options for the similarity measure. Here we choose cosine
    "user_based": False,  # item-based
}

model = KNNBasic(k=N_NEIGHBORS, sim_options=sim_options)
model.fit(train)

# save similiarity matrix to csv file
sim_matrix = model.compute_similarities()
sim_matrix_df = pd.DataFrame(sim_matrix)

raw_iids = [model.trainset.to_raw_iid(inner_id) for inner_id in range(sim_matrix_df.shape[0])]
sim_matrix_df.columns = raw_iids
sim_matrix_df.index = raw_iids
sim_matrix_df.to_csv('data/sim_matrix.csv', index=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [49]:
# Calculate the top N similar items for each item
top_n_neighbors = {}
for i, inner_id in enumerate(model.trainset.all_items()):
    raw_id = model.trainset.to_raw_iid(inner_id)
    neighbors = model.get_neighbors(inner_id, k=N_NEIGHBORS)
    top_n_neighbors[raw_id] = [model.trainset.to_raw_iid(neighbor) for neighbor in neighbors]

# save top_n_neighbors into a Json file
with open('data/top_n_neighbors.json', 'w', encoding='utf-8') as f:
    json.dump(top_n_neighbors, f, ensure_ascii=False, indent=4)

# load json file
top_n_neighbors = json.load(open('data/top_n_neighbors.json', 'r', encoding='utf-8'))

In [50]:

def recommend(user_id: str, top_n: int = 10):
    """
    recommend
    :param user_id: 
    :param top_n: 
    :return: 
    """
    game_ids = [game['app_id'] for game in games]
    # Get user rated items
    user_items = reviews_df[reviews_df['userID'] == user_id]['itemID'].tolist()
    if len(user_items) == 0:
        # If user has not rated any item 
        sorted_games = sorted(games, key=lambda x: x['rating'], reverse=True)
        return [game['app_id'] for game in sorted_games[:top_n]]
        #choices = rd.choices(sorted_games, k=top_n)
        #return [game['app_id'] for game in choices]
    # Get the top N similar items to the item rated by the users
    top_n_items = []
    for user_item in user_items:
        neighbor_list = top_n_neighbors.get(user_item, [])
        for neighbor in neighbor_list:
            if neighbor not in user_items and neighbor in game_ids:
                top_n_items.append(neighbor)
    top_n_items = list(set(top_n_items))
    return top_n_items[:top_n]
    #choices = rd.choices(top_n_items, k=top_n)
    #return choices


right now we recommend top-n games, we can also use the function random.choices() to randomly pick n recommendartions. 
random.choices(top_n_items, k=top_n) 


In [52]:
# testing
test_user_id = '3635265'
game_dict = {game['app_id']: game for game in games}
recommend_list = recommend(test_user_id)
print('[Recommended Games]')
for item in reviews_df[reviews_df['userID'] == test_user_id]['itemID'].tolist():
    print(item)
print('[Model Recommendation List]')
for item in recommend_list:
    print(item)

[Recommended Games]
374320
271590
578080
292030
359550
275850
291550
381210
1091500
1172620
431960
252490
230410
239140
444090
49520
322330
242760
251570
550
730
221100
489830
346110
582010
570
1085660
4000
1172470
304930
1063730
440
377160
[Model Recommendation List]
1174180
620
218620
367520
413150
236390
945360
892970
1245620
227300
